In [2]:
import scipy.io
import numpy as np

In [3]:
mat_data = scipy.io.loadmat('./data_preprocessed_matlab/s01.mat') 
original_data = mat_data['data']
original_label = mat_data['labels']

# (40, 40, 8064) 40试验，40通道（前32为eeg），63s（前3秒没用） x 128Hz = 8064
print("original_data形状：", original_data.shape)
# (40, 4) 40试验，4个评价指标
print("original_label形状：", original_label.shape)

original_data形状： (40, 40, 8064)
original_label形状： (40, 4)


In [4]:
'''
    处理eeg，从(40, 40, 8064)变成(40, 32, 7680)
'''
# 前32通道为eeg，去除前3秒
sliced_data = original_data[:, :32, 384:] 
print("sliced_data形状：", sliced_data.shape) # (40, 32, 7680)

eeg_data = sliced_data

sliced_data形状： (40, 32, 7680)


In [13]:
'''
    处理labels，从(40, 4)变成(800,)
'''
valence = original_label[:,0]
arousal = original_label[:,1]
# HAHV--1, LAHV--2, LALV--3, HALV--4 完全按照象限来分
VA_labels = np.where((valence > 5) & (arousal > 5), 0,
          np.where((valence >= 5) & (arousal < 5), 1,
            np.where((valence < 5) & (arousal < 5), 2, 3)))
print("V:", valence)
print("A:", arousal)
print(VA_labels)

# 将 VA_labels 从 40 扩展到 800，VA_labels中的一个数复制20次
num_segments = sliced_data.shape[2] // segment_size # 7680/3/128 = 20
expanded_VA_labels = np.repeat(VA_labels, num_segments)
print(expanded_VA_labels.shape) # (800,)
labels = expanded_VA_labels 
# print(labels)

V: [7.71 8.1  8.58 4.94 6.96 8.27 7.44 7.32 4.04 1.99 2.99 2.71 1.95 4.18
 3.17 6.81 2.46 7.23 7.17 8.26 9.   7.09 8.15 7.04 8.86 7.28 7.35 3.88
 1.36 2.08 3.03 2.28 3.81 2.28 2.06 2.9  2.31 3.33 3.24 5.1 ]
A: [7.6  7.31 7.54 6.01 3.92 3.92 3.73 2.55 3.29 4.86 2.36 2.77 3.12 2.24
 8.08 7.44 6.91 7.15 8.   7.91 7.95 2.08 3.01 7.09 7.21 7.27 6.95 3.35
 2.27 2.99 8.14 8.   3.85 7.09 8.15 6.92 6.88 7.18 6.18 7.12]
[0 0 0 3 1 1 1 1 2 2 2 2 2 2 3 0 3 0 0 0 0 1 1 0 0 0 0 2 2 2 3 3 2 3 3 3 3
 3 3 0]
(800,)


'''
    保存数据
'''
np.save('./EEGData/s01_eeg.npy', eeg_data) # (40, 32, 7680)
np.save('./EEGData/s01_labels.npy', labels) # (800,)

In [10]:
import pandas as pd
import mne
from mne import io
from mne.datasets import sample
from mne.preprocessing import ICA
import os

sfreq = 128  # 采样率为128Hz
channels = 32
samples = 384
num_each_trial = 20
ch_names = ['Fp1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5', 'P7', 'P3', 
            'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 
            'FC6', 'FC2', 'F4', 'F8', 'AF4', 'FP2', 'Fz', 'Cz']
ch_types = ['eeg'] * channels

data_list = []
eeg_data_segments = np.split(eeg_data, 40, axis=0) # (40, 32, 7680)
index = 0
for segment in eeg_data_segments:
    index = index + 1
    print("wzt index", index)
    segment_2d = segment.reshape(-1, channels).T
    print("wzt segment_2d", segment_2d.shape) # (32, 7680)
    # 创建MNE的Raw对象
    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)
    raw = mne.io.RawArray(segment_2d, info=info)
    # 滤波
    raw.filter(l_freq=1.0, h_freq=50.0)
    # 创建ICA对象并拟合数据
    ica = ICA(n_components=channels, random_state=0, max_iter=1000)  # 调整参数
    ica.fit(raw)
    # 应用ICA滤波
    ica.exclude = []  # 将排除的独立成分列表设置为空
    ica.apply(raw)
    # 获取滤波后的数据
    data = raw.get_data().T # (7680, 32)

    # 将数据调整为每个trial的形状
    data = data[:num_each_trial * samples, :]
    # 重新将数据分成每个trial的形状
    data = data.reshape(num_each_trial, samples, channels)
    # 添加数据到列表中
    data_list.append(data)

# 将数据列表转换为numpy数组并按顺序连接
data_array = np.concatenate(data_list, axis=0) # (800, 384, 32)
# 交换位置
data_array = np.swapaxes(data_array, 1, 2) # (800, 32, 384)
print(data_array.shape) 
eeg_data = data_array

wzt index 1
wzt segment_2d (32, 7680)
Creating RawArray with float64 data, n_channels=32, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 423 samples (3.305 s)

Fitting ICA to data using 32 channels (please be patient, this may take a while)
Selecting by number: 32 components


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Fitting ICA took 4.5s.
Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA components
wzt data (7680, 32)
wzt index 2
wzt segment_2d (32, 7680)
Creating RawArray with float64 data, n_channels=32, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 423 samples (3.305 s)



/tmp/ipykernel_170475/2410262094.py:32: RuntimeWarning: Using n_components=32 (resulting in n_components_=32) may lead to an unstable mixing matrix estimation because the ratio between the largest (4.7) and smallest (3.3e-13) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 27
  ica.fit(raw)


Fitting ICA to data using 32 channels (please be patient, this may take a while)
Selecting by number: 32 components


...

Selecting by number: 32 components
Fitting ICA took 3.5s.
Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA components
wzt data (7680, 32)
wzt index 40
wzt segment_2d (32, 7680)
Creating RawArray with float64 data, n_channels=32, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 423 samples (3.305 s)

Fitting ICA to data using 32 channels (plea

/tmp/ipykernel_170475/2410262094.py:32: RuntimeWarning: Using n_components=32 (resulting in n_components_=32) may lead to an unstable mixing matrix estimation because the ratio between the largest (4.8) and smallest (4.2e-13) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 27
  ica.fit(raw)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Selecting by number: 32 components
Fitting ICA took 5.1s.
Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA components


/tmp/ipykernel_170475/2410262094.py:32: RuntimeWarning: Using n_components=32 (resulting in n_components_=32) may lead to an unstable mixing matrix estimation because the ratio between the largest (4.4) and smallest (4.3e-13) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 27
  ica.fit(raw)


wzt data (7680, 32)
(800, 32, 384)


In [14]:
'''
    保存数据
'''
np.save('./EEGData/s01_eeg.npy', eeg_data) # (800, 32, 384)
np.save('./EEGData/s01_labels.npy', labels) # (800,)